<a href="https://colab.research.google.com/github/anahimaciel/Networks-in-cities/blob/main/RadiusCalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import geopy.distance

In [ ]:
city= "Franca"

# Prep

In [ ]:
#read df
df = pd.read_csv("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/filtered_cdr.txt",usecols=[1,2,3,4,5])
df

,date,time,user_from,user_to,antenna
0,2013-04-16,20:31:37,05F984397ACBD129F07AB7CCEED88C4B,A3B394659A388B22AB658C4E0390EB99,49719
1,2013-04-16,22:41:21,CDC9D4CBB5BFC2A9B3C6E13CBC51F719,E8BDE3ADDCFDC666AA4369C48C118F89,49527
2,2013-04-16,10:56:15,64BD89FE6D5AFDCAA21235A47E9D4F20,95AFF2AA53E24F78396206475E792AB3,26367
3,2013-04-16,18:17:40,64BD89FE6D5AFDCAA21235A47E9D4F20,CF8D2CF8FC54D4B667EEA38DF1862B46,26367
4,2013-04-16,12:34:05,A0DFB562CB49E2082F4B39F99639AABC,223C2B3D5931DCF4F27C151FDFE0D2E7,49549
...,...,...,...,...,...
279295,2013-04-03,13:50:12,2E2BF4B0329F18A055B371232E0A01B7,4A09B2CE09AFAFB9E810E263B70F4DFC,49608
279296,2013-04-03,15:20:12,BEA61995FF18229688F57437CE07F8C8,B583C07AA0CE285E509531A7C36B3499,49567
279297,2013-04-03,13:28:41,2E2BF4B0329F18A055B371232E0A01B7,0F28FDAB3455E0D74D5EF1E55F000E15,49608
279298,2013-04-03,14:53:09,BEA61995FF18229688F57437CE07F8C8,B52CE2BDC499A6EE7EF340FBA2105295,49567


In [ ]:
#read antennas files
map = pd.read_csv("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/antennas_"+city+".txt", sep=';')
map

,CELLID,UF,DDD,LAT,LONG,CITY,MUNICIPALITY
0,12967,SP,16,-20.50771,-47.42401,Franca,Região Imediata de Franca
1,12968,SP,16,-20.50771,-47.42401,Franca,Região Imediata de Franca
2,48207,SP,16,-20.56161,-47.37077,Franca,Região Imediata de Franca
3,49558,SP,16,-20.53421,-47.40857,Franca,Região Imediata de Franca
4,49637,SP,16,-20.55239,-47.44844,Franca,Região Imediata de Franca
...,...,...,...,...,...,...,...
70,49659,SP,16,-20.58959,-47.37296,Franca,Região Imediata de Franca
71,12959,SP,16,-20.48961,-47.39745,Franca,Região Imediata de Franca
72,49569,SP,16,-20.53800,-47.41717,Franca,Região Imediata de Franca
73,49599,SP,16,-20.50482,-47.39958,Franca,Região Imediata de Franca


# Compute home locations

In [ ]:
#convert to datetime object
df['date']=pd.to_datetime(df['date'])
#create column with days of the week
df['day_week']=df['date'].dt.day_name()

#create dataframe with calls in weekends and nights
night_df=df[(df['day_week']=='Saturday') | (df['day_week']=='Sunday') | (df['time']> '19:00:00') | (df['time']< '06:00:00')]
night_df

,date,time,user_from,user_to,antenna,day_week
0,2013-04-16,20:31:37,05F984397ACBD129F07AB7CCEED88C4B,A3B394659A388B22AB658C4E0390EB99,49719,Tuesday
1,2013-04-16,22:41:21,CDC9D4CBB5BFC2A9B3C6E13CBC51F719,E8BDE3ADDCFDC666AA4369C48C118F89,49527,Tuesday
8,2013-04-16,21:25:11,64BD89FE6D5AFDCAA21235A47E9D4F20,016CC65D53B4E0755DE47A40B9C3D1E3,26367,Tuesday
24,2013-04-16,20:03:40,05F984397ACBD129F07AB7CCEED88C4B,A3B394659A388B22AB658C4E0390EB99,49719,Tuesday
25,2013-04-16,20:53:37,CDC9D4CBB5BFC2A9B3C6E13CBC51F719,AAFDC373347683C1547DCF19B7646873,49527,Tuesday
...,...,...,...,...,...,...
279195,2013-04-03,20:33:39,A3200196E647E1DCC35A33424DDB00D2,BD7CE5EFC3D1C2A2B7B87F62DF30F0E3,49589,Wednesday
279231,2013-04-03,22:01:51,28B135512C864F59EDBCBA30FD228183,4ECB20FAF072502BE9A5B0E212E058B4,49578,Wednesday
279248,2013-04-03,20:28:08,A3200196E647E1DCC35A33424DDB00D2,2E3CC14F5CF5A25C62E0CDDDA0AB7322,49589,Wednesday
279251,2013-04-03,22:03:48,28B135512C864F59EDBCBA30FD228183,AB2F4DAAFEFC8E61344ABEFDF9DE214D,49578,Wednesday


In [ ]:
#create dataframe with data necessary to compute places of residence
residence_df=night_df.groupby(['user_from','antenna']).size().reset_index(name='time')
residence_df

,user_from,antenna,time
0,00192A050D3563F1879A8F6CEE69CE84,49618,1
1,00192A050D3563F1879A8F6CEE69CE84,49657,4
2,0019CBAA3A6B25642585B321F31E733E,49578,1
3,001F6105804D4BBAC07EE5022E9CC4AB,49548,2
4,001F6105804D4BBAC07EE5022E9CC4AB,49587,4
...,...,...,...
11703,FFF57458D4812B13090843E812854ABC,49678,2
11704,FFF57458D4812B13090843E812854ABC,49688,1
11705,FFF57458D4812B13090843E812854ABC,49689,2
11706,FFF57458D4812B13090843E812854ABC,49698,1


In [ ]:
#compute places of residence
#drop locations other than the first
residence_df=residence_df.sort_values(['user_from','time'],ascending=[True,False]).drop_duplicates('user_from')
#only keep users whose first location complies with certain parameters (more than half of the calls of the user)
number_calls_map=residence_df['user_from'].map(df.groupby('user_from').size())
filter=(residence_df['time']>number_calls_map/2)
residence_df=residence_df[filter]

#include geographical location of antennas in residence dataframe
map=map.rename(columns={"CELLID": "antenna"})
residence_df=pd.merge(residence_df,map,on="antenna",how="left")

#delete unnecessary columns
residence_df=residence_df.drop(['time','UF','DDD','CITY','MUNICIPALITY'],axis=1)

residence_df.sort_values('antenna')

,user_from,antenna,LAT,LONG
334,356B75C31EDB18CB17B0158D854AEB91,12927,-20.50970,-47.36730
1181,C51D709C6288DF95F9B755748E29AD1B,12927,-20.50970,-47.36730
1063,B191CB12DDC46FE2C4C70C9D9B219CA8,12927,-20.50970,-47.36730
725,76D5D5F6BCF3A2C2F226338A882CFA1E,12927,-20.50970,-47.36730
1183,C546616C699E61989F0FDAB7EEA5319B,12927,-20.50970,-47.36730
...,...,...,...,...
1360,E3EF3500D3FB7701C96E271F191C4FD6,49719,-20.57316,-47.38403
389,3D6EC2E45EA0C090F1B1CDE0A361D6FF,49719,-20.57316,-47.38403
1267,D41549ABB3ED544CDC8323A677B12C1D,49719,-20.57316,-47.38403
757,7BA47BF0681D91CD0B1A318A0DAF2276,49719,-20.57316,-47.38403


In [ ]:
#extract from df users whose place of residence were not found (useful for next step)
filter=df['user_from'].isin(residence_df['user_from']) & df['user_to'].isin(residence_df['user_from'])
filtered_df=df[filter]
filtered_df

,date,time,user_from,user_to,antenna,day_week
11101,2013-04-16,22:42:29,8BD40735D89DEFDE48F595BDDDC6F855,A1996CAE85F123D2EF57D2BABF0AE7D4,49699,Tuesday
20853,2013-04-19,22:17:54,E9506C18D3DF912B85023BFCB6B7E98E,76EE0DE24F32E1446B830ADD5825EAF8,48208,Friday
26547,2013-04-02,21:48:51,2679B4659929A312B80AE51A5EB48EF4,F10C4660D57D55CBB4BA4CA463C833FA,49619,Tuesday
37242,2013-03-29,15:26:46,14D939FD2B2DB313CFE3B86C5010EF57,F439191B4FA22631344A3F5383507DF3,49618,Friday
38384,2013-04-05,23:22:31,8B006C8CF6B783331C3243A4D1E1FB76,818CAEC48EEB47DC7D2231B16A7BE347,49598,Friday
...,...,...,...,...,...,...
258051,2013-04-10,20:11:35,D44F93ADB13A8019C31C5B9F9270B719,0AB16A774AF06D243E818EE887B57200,49619,Wednesday
269102,2013-04-03,20:56:17,D44F93ADB13A8019C31C5B9F9270B719,0AB16A774AF06D243E818EE887B57200,49719,Wednesday
269486,2013-04-03,18:56:05,D44F93ADB13A8019C31C5B9F9270B719,0AB16A774AF06D243E818EE887B57200,49619,Wednesday
269515,2013-04-03,20:57:47,D44F93ADB13A8019C31C5B9F9270B719,0AB16A774AF06D243E818EE887B57200,49619,Wednesday


# Calculate average radius

In [ ]:
#dicts with coordinates
dict_lat = residence_df.set_index('user_from')['LAT'].to_dict()
dict_long = residence_df.set_index('user_from')['LONG'].to_dict()

#function to calculate distance between residences of caller and receiver for each call
def get_distance(row):
    coords_caller=(dict_lat[row.user_from],dict_long[row.user_from])                #deixar mais rapido: agrupar por pessoa q ta ligando (n sei se ajudaria mt)
    coords_receiver=(dict_lat[row.user_to],dict_long[row.user_to])
    return geopy.distance.distance(coords_caller,coords_receiver).km

#apply function to filtered_df
filtered_df['distance'] = filtered_df.apply(get_distance, axis=1)
filtered_df

<ipython-input-10-bd0efc1160f5>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['distance'] = filtered_df.apply(get_distance, axis=1)


,date,time,user_from,user_to,antenna,day_week,distance
11101,2013-04-16,22:42:29,8BD40735D89DEFDE48F595BDDDC6F855,A1996CAE85F123D2EF57D2BABF0AE7D4,49699,Tuesday,0.000000
20853,2013-04-19,22:17:54,E9506C18D3DF912B85023BFCB6B7E98E,76EE0DE24F32E1446B830ADD5825EAF8,48208,Friday,3.106113
26547,2013-04-02,21:48:51,2679B4659929A312B80AE51A5EB48EF4,F10C4660D57D55CBB4BA4CA463C833FA,49619,Tuesday,0.000000
37242,2013-03-29,15:26:46,14D939FD2B2DB313CFE3B86C5010EF57,F439191B4FA22631344A3F5383507DF3,49618,Friday,0.000000
38384,2013-04-05,23:22:31,8B006C8CF6B783331C3243A4D1E1FB76,818CAEC48EEB47DC7D2231B16A7BE347,49598,Friday,0.000000
...,...,...,...,...,...,...,...
258051,2013-04-10,20:11:35,D44F93ADB13A8019C31C5B9F9270B719,0AB16A774AF06D243E818EE887B57200,49619,Wednesday,0.000000
269102,2013-04-03,20:56:17,D44F93ADB13A8019C31C5B9F9270B719,0AB16A774AF06D243E818EE887B57200,49719,Wednesday,0.000000
269486,2013-04-03,18:56:05,D44F93ADB13A8019C31C5B9F9270B719,0AB16A774AF06D243E818EE887B57200,49619,Wednesday,0.000000
269515,2013-04-03,20:57:47,D44F93ADB13A8019C31C5B9F9270B719,0AB16A774AF06D243E818EE887B57200,49619,Wednesday,0.000000


In [ ]:
#drop calls to the same receiver (nao sei pq)
groups=filtered_df.groupby(['user_from','user_to'])
aux=groups.first()
unique_calls=len(aux)
aux=aux.groupby('user_from')['distance'].sum().to_frame(name='total_distance_without_alpha')
non_zero_df=aux[aux['total_distance_without_alpha']!=0]

#calculate average radius for each user
radii=filtered_df.groupby("user_from").agg({'distance':['sum','size'],'user_to':'nunique'})
radii.columns=['total_distance','n_calls','contacts']
radii['avg_dist_by_calls']=radii['total_distance']/radii['n_calls']
radii['avg_radius']=radii['total_distance']/radii['contacts']
radii['total_distance_without_alpha']=aux['total_distance_without_alpha']
radii['avg_radius_without_alpha']=aux['total_distance_without_alpha']/radii['contacts']
radii

,total_distance,n_calls,contacts,avg_dist_by_calls,avg_radius,total_distance_without_alpha,avg_radius_without_alpha
user_from,,,,,,,
0030CBFB792766A3BEE8B65AACFE88E5,0.000000,1,1,0.000000,0.000000,0.000000,0.000000
02241A1E7098FB493A2A8D924E99C116,0.000000,2,1,0.000000,0.000000,0.000000,0.000000
0340C87B4A1FAB2926C8BFCCE2090C2E,6.846421,1,1,6.846421,6.846421,6.846421,6.846421
05109233CEE01597FDE1845C3BF61A76,12.393767,4,1,3.098442,12.393767,3.098442,3.098442
0AB16A774AF06D243E818EE887B57200,0.000000,1,1,0.000000,0.000000,0.000000,0.000000
14D939FD2B2DB313CFE3B86C5010EF57,0.000000,2,1,0.000000,0.000000,0.000000,0.000000
1C7872F0559ED08D46CFB0F57D5BF9D7,0.000000,1,1,0.000000,0.000000,0.000000,0.000000
2329F3F87406B19DB1203411EE593F92,0.000000,2,1,0.000000,0.000000,0.000000,0.000000
2679B4659929A312B80AE51A5EB48EF4,0.000000,5,1,0.000000,0.000000,0.000000,0.000000


In [ ]:
#results
with open("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/results.txt",'a') as file:
  file.truncate(0)
  for i in radii.columns:
    results_df=radii.agg({i:['max','min','mean']})
    file.write(results_df.to_string(index=False)+"\n")
    print(results_df)

      total_distance
max        12.393767
min         0.000000
mean        1.799084
      n_calls
max       8.0
min       1.0
mean      2.0
      contacts
max       2.00
min       1.00
mean      1.04
      avg_dist_by_calls
max            6.846421
min            0.000000
mean           1.059737
      avg_radius
max    12.393767
min     0.000000
mean    1.743577
      total_distance_without_alpha
max                       6.846421
min                       0.000000
mean                      1.115243
      avg_radius_without_alpha
max                   6.846421
min                   0.000000
mean                  1.059737


In [ ]:
#saving demography data
with open("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/demo2.txt",'a') as file:
  file.truncate(0)
  file.write("users with residence: "+str(len(residence_df))+"\n")
  file.write("calls by users with residence: "+str(len(filtered_df))+"\n")
  file.write("calls to unique contacts by users with residence: "+str(len(aux))+"\n")
  file.write("calls to unique contacts by users with residence with distance != 0: "+ str(len(non_zero_df))+"\n")


In [ ]:
#saving results
residence_df.to_csv("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/home_locations.txt")
filtered_df.to_csv("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/filtered_cdr_2.txt")
radii.to_csv("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/radii.txt")

#for the networks
edges=groups.size().reset_index(name='weight')
edges.to_csv("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/edges.txt")